# 🧬 NonBDNAFinder — RAM-Adaptive Analysis & Rigorous Report Generator

**Author:** Dr. Venkata Rajesh Yella | **License:** MIT | **Version:** 2024.2

This notebook:
1. **Detects available RAM** and automatically adjusts chunk size, worker count, and execution mode.
2. **Runs all 9 Non-B DNA detectors** (G-Quadruplex, Z-DNA, Curved DNA, R-Loop, Slipped DNA, Cruciform, Triplex, i-Motif, A-philic DNA) plus hybrid and cluster detection.
3. **Generates a rigorous publication-quality report** (statistics tables, plots, per-class breakdowns, and multi-format exports) with a **single cell** execution.

---
### How to use
1. Run **Cell 1** once to install/verify dependencies.
2. Edit **Cell 2** to point to your FASTA file and set any desired options.
3. Run **Cell 3** to preview the detected system resources and the adaptive plan.
4. Run **Cell 4** (the *single analysis + report cell*) — it does everything: analysis, statistics, plots, and exports.

> **Tip:** You can also *Run All* (`Kernel → Restart & Run All`) for a fully automated pipeline.

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1 — Dependency check & path setup
# Run once.  Safe to re-run.
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, importlib

# Make sure the repository root is on PYTHONPATH so local modules resolve
_REPO_ROOT = os.path.abspath(os.path.dirname("__file__"))
if _REPO_ROOT not in sys.path:
    sys.path.insert(0, _REPO_ROOT)

_REQUIRED = [
    ("psutil",     "psutil>=5.8.0"),
    ("pandas",     "pandas>=1.3.0"),
    ("numpy",      "numpy>=1.21.0"),
    ("matplotlib", "matplotlib>=3.5.0"),
    ("seaborn",    "seaborn>=0.11.0"),
    ("openpyxl",   "openpyxl>=3.0.0"),
]

_missing = [pkg for mod, pkg in _REQUIRED if importlib.util.find_spec(mod) is None]
if _missing:
    print(f"Installing missing packages: {_missing}")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", *_missing, "-q"])

print("✅ All dependencies satisfied.")
print(f"   Python : {sys.version.split()[0]}")
import pandas as pd, numpy as np
print(f"   pandas : {pd.__version__}   numpy : {np.__version__}")

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 2 — User configuration
# Edit the values in this cell to match your environment.
# ─────────────────────────────────────────────────────────────────────────────

# ── Input ────────────────────────────────────────────────────────────────────
# Path to the FASTA / FNA file to analyse.
# Accepts single-sequence or multi-sequence FASTA files.
FASTA_FILE = "example.fasta"          # ← change to your file path

# ── Output directory ─────────────────────────────────────────────────────────
# All exports (CSV, BED, JSON, Excel) and the HTML report are written here.
OUTPUT_DIR = "notebook_reports"        # ← change if desired

# ── Motif classes ─────────────────────────────────────────────────────────────
# Set to None to run ALL 9 detectors (recommended).
# Or pass a list of class names, e.g.:
#   ENABLED_CLASSES = ["G-Quadruplex", "Z-DNA", "Cruciform"]
ENABLED_CLASSES = None

# ── RAM override (optional) ───────────────────────────────────────────────────
# Leave as None to auto-detect.  Override with bytes, e.g. 2_000_000_000 for 2 GB.
RAM_OVERRIDE_BYTES = None

# ── Export formats ───────────────────────────────────────────────────────────
EXPORT_CSV   = True
EXPORT_BED   = True
EXPORT_JSON  = True
EXPORT_EXCEL = True

# ── Report verbosity ─────────────────────────────────────────────────────────
# 'standard' = summary tables + 4 plots
# 'full'     = standard + per-class breakdowns + co-occurrence heatmap
REPORT_MODE = "full"

print("✅ Configuration set.")
print(f"   FASTA_FILE     : {FASTA_FILE}")
print(f"   OUTPUT_DIR     : {OUTPUT_DIR}")
print(f"   ENABLED_CLASSES: {ENABLED_CLASSES or 'all 9 detectors'}")
print(f"   REPORT_MODE    : {REPORT_MODE}")

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 3 — System resource detection & adaptive plan preview
# Read-only: shows what the analysis cell will use.
# ─────────────────────────────────────────────────────────────────────────────
from Utilities.system_resource_inspector import SystemResourceInspector
from Utilities.adaptive_chunk_planner    import AdaptiveChunkPlanner

_inspector = SystemResourceInspector()
_total_ram   = _inspector.get_total_ram()
_avail_ram   = _inspector.get_available_ram()
_ram_budget  = RAM_OVERRIDE_BYTES if RAM_OVERRIDE_BYTES else _inspector.get_memory_budget()
_cpu_count   = _inspector.get_cpu_count()
_disk_free   = _inspector.get_available_disk(".")

# Estimate genome length from file size (rough proxy: ~1 byte per bp in plain FASTA)
_fasta_size  = os.path.getsize(FASTA_FILE) if os.path.exists(FASTA_FILE) else 1_000_000
_est_genome  = max(_fasta_size, 1_000)     # conservative lower bound

_planner = AdaptiveChunkPlanner()
_plan    = _planner.plan(
    genome_length = _est_genome,
    ram_budget    = _ram_budget,
    cpu_count     = _cpu_count,
)

print("━" * 58)
print("  System Resource Report")
print("━" * 58)
print(f"  Total RAM      : {_total_ram  / 1e9:.2f} GB")
print(f"  Available RAM  : {_avail_ram  / 1e9:.2f} GB")
print(f"  Memory budget  : {_ram_budget / 1e9:.2f} GB"
      + ("  [user override]" if RAM_OVERRIDE_BYTES else ""))
print(f"  CPU cores      : {_cpu_count}")
print(f"  Disk free (.)  : {_disk_free  / 1e9:.2f} GB")
print()
print("━" * 58)
print("  Adaptive Execution Plan")
print("━" * 58)
print(f"  Chunk size     : {_plan['chunk_size']:,} bp")
print(f"  Overlap        : {_plan['overlap']:,} bp")
print(f"  Workers        : {_plan['workers']}")
print(f"  Mode           : {_plan['mode']}")
print("━" * 58)

if _ram_budget < 1_000_000_000:
    print("⚠️  Low RAM detected (<1 GB budget): using conservative disk_stream mode.")
elif _ram_budget < 4_000_000_000:
    print("ℹ️  Moderate RAM (1–4 GB budget): using disk_stream with 2 workers.")
else:
    print("✅ Ample RAM (≥4 GB budget): using hybrid mode with up to 4 workers.")

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# CELL 4 — SINGLE-CELL ANALYSIS + RIGOROUS REPORT GENERATOR
#
# • Reads the FASTA file specified in Cell 2.
# • Automatically adapts chunk size / workers / mode to available RAM (Cell 3).
# • Runs all enabled detectors and hybrid/cluster post-processing.
# • Computes publication-quality statistics.
# • Renders 4–6 plots inline.
# • Exports results in every enabled format.
# • Writes a self-contained HTML report to OUTPUT_DIR.
#
# ══  Run this cell once after setting Cell 2.  ══════════════════════════════
# ═════════════════════════════════════════════════════════════════════════════

import gc
import json
import time
import datetime
import warnings
import textwrap
from collections import Counter, defaultdict
from pathlib import Path
from IPython.display import display, HTML, Image

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

from Utilities.system_resource_inspector import SystemResourceInspector
from Utilities.adaptive_chunk_planner    import AdaptiveChunkPlanner
from Utilities.nonbscanner               import analyze_sequence as _nbf_analyze
from Utilities.utilities                 import (
    read_fasta_file, export_to_csv, export_to_bed,
    export_to_json, export_to_excel,
    calculate_motif_statistics,
)

# ── 0. Prepare output directory ───────────────────────────────────────────────
_OUT = Path(OUTPUT_DIR)
_OUT.mkdir(parents=True, exist_ok=True)
_RUN_TS = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
_BASE    = _OUT / _RUN_TS
_BASE.mkdir(parents=True, exist_ok=True)

print(f"📂 Output directory : {_BASE}")

# ── 1. Resolve adaptive plan ──────────────────────────────────────────────────
_insp   = SystemResourceInspector()
_budget = RAM_OVERRIDE_BYTES if RAM_OVERRIDE_BYTES else _insp.get_memory_budget()
_cpus   = _insp.get_cpu_count()
_plan   = AdaptiveChunkPlanner().plan(
    genome_length = os.path.getsize(FASTA_FILE) if os.path.exists(FASTA_FILE) else 1_000_000,
    ram_budget    = _budget,
    cpu_count     = _cpus,
)
CHUNK_SIZE    = _plan["chunk_size"]
CHUNK_OVERLAP = _plan["overlap"]
N_WORKERS     = _plan["workers"]
EXEC_MODE     = _plan["mode"]

print(f"⚙️  Adaptive plan   : chunk={CHUNK_SIZE:,} bp, overlap={CHUNK_OVERLAP:,} bp, "
      f"workers={N_WORKERS}, mode={EXEC_MODE}")

# ── 2. Parse FASTA ────────────────────────────────────────────────────────────
if not os.path.exists(FASTA_FILE):
    raise FileNotFoundError(f"FASTA file not found: {FASTA_FILE!r}")

_sequences = read_fasta_file(FASTA_FILE)   # returns {name: sequence}
if not _sequences:
    raise ValueError(f"No sequences parsed from {FASTA_FILE!r}")

print(f"🔬 Sequences loaded : {len(_sequences)} sequence(s)")
for _sn, _sq in list(_sequences.items())[:5]:
    print(f"     {_sn[:60]:<60}  {len(_sq):>12,} bp")
if len(_sequences) > 5:
    print(f"     … and {len(_sequences) - 5} more")

# ── 3. Run detection ──────────────────────────────────────────────────────────
print()
print("🚀 Running motif detection …")
_t0          = time.perf_counter()
_all_motifs  = []
_seq_stats   = {}   # per-sequence summary

for _seq_idx, (_seq_name, _seq) in enumerate(_sequences.items(), 1):
    _seq_len = len(_seq)
    print(f"   [{_seq_idx}/{len(_sequences)}] {_seq_name[:50]} ({_seq_len:,} bp)", end="", flush=True)
    _t_seq = time.perf_counter()

    _motifs = _nbf_analyze(
        sequence         = _seq,
        sequence_name    = _seq_name,
        use_chunking     = True,
        chunk_size       = CHUNK_SIZE,
        chunk_overlap    = CHUNK_OVERLAP,
        use_parallel_chunks = (EXEC_MODE == "hybrid"),
        enabled_classes  = ENABLED_CLASSES,
    )

    _elapsed = time.perf_counter() - _t_seq
    print(f"  → {len(_motifs):,} motifs  ({_elapsed:.1f} s)")

    _all_motifs.extend(_motifs)
    _seq_stats[_seq_name] = {
        "length": _seq_len,
        "motifs": len(_motifs),
        "elapsed_s": round(_elapsed, 2),
        "throughput_kbps": round(_seq_len / max(_elapsed, 1e-6) / 1000, 1),
    }
    gc.collect()

_total_elapsed = time.perf_counter() - _t0
print(f"\n✅ Detection complete: {len(_all_motifs):,} total motifs in {_total_elapsed:.1f} s")

# ── 4. Build DataFrame ────────────────────────────────────────────────────────
_df = pd.DataFrame(_all_motifs) if _all_motifs else pd.DataFrame()

# Ensure canonical columns exist
for _col, _default in [
    ("Class",         "Unknown"),
    ("Subclass",      "Other"),
    ("Start",         0),
    ("End",           0),
    ("Length",        0),
    ("Score",         0.0),
    ("Strand",        "+"),
    ("Sequence_Name", ""),
]:
    if _col not in _df.columns:
        _df[_col] = _default

if "Length" in _df.columns and _df["Length"].eq(0).any() and "Start" in _df.columns:
    _mask = _df["Length"] == 0
    _df.loc[_mask, "Length"] = (_df.loc[_mask, "End"] - _df.loc[_mask, "Start"]).clip(lower=0)

# ── 5. Compute statistics ─────────────────────────────────────────────────────
_total_bp      = sum(len(s) for s in _sequences.values())
_total_motifs  = len(_df)
_n_seqs        = len(_sequences)

# Per-class counts
_class_counts  = _df["Class"].value_counts().to_dict() if not _df.empty else {}
_subclass_counts = _df["Subclass"].value_counts().to_dict() if not _df.empty else {}

# Coverage: fraction of genome bases covered by ≥1 motif
def _compute_coverage(df, seq_len):
    """Compute base-pair coverage (fraction) using merged intervals."""
    if df.empty or seq_len == 0:
        return 0.0
    intervals = sorted(zip(df["Start"].clip(lower=1), df["End"]), key=lambda x: x[0])
    covered = 0
    cur_start = cur_end = -1
    for s, e in intervals:
        if s > cur_end:
            if cur_end >= 0:
                covered += cur_end - cur_start + 1
            cur_start, cur_end = s, e
        else:
            cur_end = max(cur_end, e)
    if cur_end >= 0:
        covered += cur_end - cur_start + 1
    return min(covered / seq_len, 1.0)

_coverage = _compute_coverage(_df, _total_bp)
_density  = (_total_motifs / (_total_bp / 1000)) if _total_bp > 0 else 0.0   # motifs / kb

# Length stats
_len_stats = {}
if not _df.empty and "Length" in _df.columns:
    _len_stats = {
        "mean":   round(_df["Length"].mean(), 1),
        "median": round(_df["Length"].median(), 1),
        "min":    int(_df["Length"].min()),
        "max":    int(_df["Length"].max()),
        "std":    round(_df["Length"].std(), 1),
    }

# Score stats
_score_stats = {}
if not _df.empty and "Score" in _df.columns:
    _score_stats = {
        "mean":   round(_df["Score"].mean(), 3),
        "median": round(_df["Score"].median(), 3),
        "min":    round(_df["Score"].min(), 3),
        "max":    round(_df["Score"].max(), 3),
    }

# Hybrid / cluster counts
_hybrid_count  = int(_class_counts.get("Hybrid", 0))
_cluster_count = int(_class_counts.get("Non-B_DNA_Clusters", 0))

# ── 6. Print summary statistics ───────────────────────────────────────────────
print()
print("═" * 62)
print("  ANALYSIS SUMMARY")
print("═" * 62)
print(f"  FASTA file        : {FASTA_FILE}")
print(f"  Run timestamp     : {_RUN_TS}")
print(f"  Sequences         : {_n_seqs}")
print(f"  Total genome (bp) : {_total_bp:,}")
print(f"  Total motifs      : {_total_motifs:,}")
print(f"  Genome coverage   : {_coverage * 100:.2f} %")
print(f"  Density (motif/kb): {_density:.3f}")
print(f"  Hybrid motifs     : {_hybrid_count:,}")
print(f"  Cluster motifs    : {_cluster_count:,}")
if _len_stats:
    print(f"  Motif length (bp) : mean={_len_stats['mean']},  "
          f"median={_len_stats['median']},  "
          f"min={_len_stats['min']},  max={_len_stats['max']}")
if _score_stats:
    print(f"  Motif score (1-3) : mean={_score_stats['mean']},  "
          f"median={_score_stats['median']},  "
          f"min={_score_stats['min']},  max={_score_stats['max']}")
print("─" * 62)
print("  Motifs per class:")
for _cls, _cnt in sorted(_class_counts.items(), key=lambda x: -x[1]):
    print(f"    {_cls:<30} {_cnt:>8,}")
print("═" * 62)

# ── 7. Visualizations ─────────────────────────────────────────────────────────
# Colour palette (Nature / colour-blind safe)
_CLASS_COLOURS = {
    "G-Quadruplex":       "#E64B35",
    "Z-DNA":              "#4DBBD5",
    "Curved_DNA":         "#00A087",
    "R-Loop":             "#3C5488",
    "Slipped_DNA":        "#F39B7F",
    "Cruciform":          "#8491B4",
    "Triplex":            "#91D1C2",
    "i-Motif":            "#DC0000",
    "A-philic_DNA":       "#7E6148",
    "Hybrid":             "#B09C85",
    "Non-B_DNA_Clusters": "#CEC4C1",
}
_DEFAULT_COLOUR = "#999999"

sns.set_theme(style="whitegrid", font_scale=1.0)
_fig_paths = []

# ── 7a. Class distribution bar chart ─────────────────────────────────────────
if _class_counts:
    _cls_df = pd.DataFrame(
        sorted(_class_counts.items(), key=lambda x: x[1], reverse=True),
        columns=["Class", "Count"]
    )
    _colours = [_CLASS_COLOURS.get(c, _DEFAULT_COLOUR) for c in _cls_df["Class"]]
    fig1, ax1 = plt.subplots(figsize=(10, 5))
    ax1.barh(_cls_df["Class"][::-1], _cls_df["Count"][::-1], color=_colours[::-1], edgecolor="none")
    ax1.set_xlabel("Motif Count", fontsize=12)
    ax1.set_title("Non-B DNA Motif Class Distribution", fontsize=14, fontweight="bold")
    for _bar, _cnt in zip(ax1.patches, _cls_df["Count"][::-1]):
        ax1.text(_bar.get_width() + 0.01 * _cls_df["Count"].max(),
                 _bar.get_y() + _bar.get_height() / 2,
                 f"{_cnt:,}", va="center", fontsize=9)
    plt.tight_layout()
    _fp1 = str(_BASE / "class_distribution.png")
    fig1.savefig(_fp1, dpi=150, bbox_inches="tight")
    plt.close(fig1)
    _fig_paths.append(("Class Distribution", _fp1))
    display(Image(_fp1))

# ── 7b. Positional density histogram ─────────────────────────────────────────
if not _df.empty and _total_bp > 0:
    _N_BINS = min(200, max(50, _total_bp // 10_000))
    _edges  = np.linspace(1, _total_bp, _N_BINS + 1)
    _starts = _df["Start"].values.clip(1, _total_bp)
    _density_hist, _ = np.histogram(_starts, bins=_edges)
    _centres = (_edges[:-1] + _edges[1:]) / 2

    fig2, ax2 = plt.subplots(figsize=(12, 4))
    ax2.fill_between(_centres, _density_hist, color="#1f77b4", alpha=0.75, linewidth=0)
    ax2.set_xlabel("Genomic Position (bp)", fontsize=12)
    ax2.set_ylabel("Motif Count / Bin", fontsize=12)
    ax2.set_title("Non-B DNA Motif Positional Density", fontsize=14, fontweight="bold")
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x/1e6:.1f} Mb" if x >= 1e6 else f"{x/1e3:.0f} kb"))
    plt.tight_layout()
    _fp2 = str(_BASE / "positional_density.png")
    fig2.savefig(_fp2, dpi=150, bbox_inches="tight")
    plt.close(fig2)
    _fig_paths.append(("Positional Density", _fp2))
    display(Image(_fp2))

# ── 7c. Motif length distribution ────────────────────────────────────────────
if not _df.empty and "Length" in _df.columns and _df["Length"].sum() > 0:
    _lengths = _df["Length"].clip(upper=5000)
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    ax3.hist(_lengths, bins=80, color="#ff7f0e", edgecolor="none", alpha=0.85)
    ax3.set_xlabel("Motif Length (bp)", fontsize=12)
    ax3.set_ylabel("Count", fontsize=12)
    ax3.set_title("Motif Length Distribution", fontsize=14, fontweight="bold")
    ax3.axvline(_lengths.mean(),   color="#d62728", linestyle="--", linewidth=1.5, label=f"Mean {_lengths.mean():.0f} bp")
    ax3.axvline(_lengths.median(), color="#2ca02c", linestyle=":",  linewidth=1.5, label=f"Median {_lengths.median():.0f} bp")
    ax3.legend(fontsize=9)
    plt.tight_layout()
    _fp3 = str(_BASE / "length_distribution.png")
    fig3.savefig(_fp3, dpi=150, bbox_inches="tight")
    plt.close(fig3)
    _fig_paths.append(("Length Distribution", _fp3))
    display(Image(_fp3))

# ── 7d. Score distribution (violin, per class) ───────────────────────────────
if not _df.empty and "Score" in _df.columns and _df["Score"].sum() > 0:
    _score_df = _df[["Class", "Score"]].copy()
    _top_classes = _score_df["Class"].value_counts().head(9).index.tolist()
    _score_df = _score_df[_score_df["Class"].isin(_top_classes)]
    _order = _score_df.groupby("Class")["Score"].median().sort_values(ascending=False).index

    fig4, ax4 = plt.subplots(figsize=(12, 5))
    sns.violinplot(
        data=_score_df, x="Class", y="Score", order=_order,
        palette={c: _CLASS_COLOURS.get(c, _DEFAULT_COLOUR) for c in _order},
        inner="box", cut=0, ax=ax4,
    )
    ax4.set_xlabel("", fontsize=11)
    ax4.set_ylabel("Normalised Score (1 – 3)", fontsize=11)
    ax4.set_title("Score Distribution per Motif Class", fontsize=14, fontweight="bold")
    ax4.tick_params(axis="x", rotation=30)
    plt.tight_layout()
    _fp4 = str(_BASE / "score_distribution.png")
    fig4.savefig(_fp4, dpi=150, bbox_inches="tight")
    plt.close(fig4)
    _fig_paths.append(("Score Distribution", _fp4))
    display(Image(_fp4))

# ── 7e. Co-occurrence heatmap (full mode only) ────────────────────────────────
if REPORT_MODE == "full" and not _df.empty:
    _cooc_classes = [c for c in _class_counts if c not in ("Hybrid", "Non-B_DNA_Clusters")]
    if len(_cooc_classes) >= 2:
        _BIN_SIZE = max(1000, _total_bp // 500)
        _n_bins   = max(1, _total_bp // _BIN_SIZE)
        _cooc_mat = pd.DataFrame(0, index=_cooc_classes, columns=_cooc_classes, dtype=np.int32)

        _bin_to_classes = defaultdict(set)
        for _, _row in _df[_df["Class"].isin(_cooc_classes)].iterrows():
            _b = min(int(_row["Start"]) // _BIN_SIZE, _n_bins - 1)
            _bin_to_classes[_b].add(_row["Class"])

        for _classes_in_bin in _bin_to_classes.values():
            _lst = sorted(_classes_in_bin)
            for _i in range(len(_lst)):
                for _j in range(_i + 1, len(_lst)):
                    if _lst[_i] in _cooc_mat.index and _lst[_j] in _cooc_mat.columns:
                        _cooc_mat.loc[_lst[_i], _lst[_j]] += 1
                        _cooc_mat.loc[_lst[_j], _lst[_i]] += 1

        if _cooc_mat.values.sum() > 0:
            fig5, ax5 = plt.subplots(figsize=(9, 7))
            sns.heatmap(
                _cooc_mat, annot=True, fmt="d", cmap="YlOrRd",
                linewidths=0.4, ax=ax5, cbar_kws={"shrink": 0.8},
            )
            ax5.set_title(f"Motif Co-occurrence Heatmap  (bin = {_BIN_SIZE:,} bp)",
                          fontsize=13, fontweight="bold")
            plt.tight_layout()
            _fp5 = str(_BASE / "cooccurrence_heatmap.png")
            fig5.savefig(_fp5, dpi=150, bbox_inches="tight")
            plt.close(fig5)
            _fig_paths.append(("Co-occurrence Heatmap", _fp5))
            display(Image(_fp5))

# ── 7f. Per-sequence summary bar chart (full mode, multi-FASTA) ───────────────
if REPORT_MODE == "full" and len(_sequences) > 1:
    _seq_names = list(_seq_stats.keys())
    _seq_motif_counts = [_seq_stats[s]["motifs"] for s in _seq_names]
    fig6, ax6 = plt.subplots(figsize=(max(8, len(_seq_names) * 0.9 + 2), 4))
    ax6.bar(range(len(_seq_names)), _seq_motif_counts, color="#4DBBD5", edgecolor="none")
    ax6.set_xticks(range(len(_seq_names)))
    ax6.set_xticklabels([s[:30] for s in _seq_names], rotation=45, ha="right", fontsize=8)
    ax6.set_ylabel("Motif Count", fontsize=11)
    ax6.set_title("Motifs per Sequence", fontsize=13, fontweight="bold")
    plt.tight_layout()
    _fp6 = str(_BASE / "per_sequence_counts.png")
    fig6.savefig(_fp6, dpi=150, bbox_inches="tight")
    plt.close(fig6)
    _fig_paths.append(("Per-sequence Counts", _fp6))
    display(Image(_fp6))

# ── 8. Export data files ──────────────────────────────────────────────────────
_export_paths = {}
if not _df.empty:
    _motif_list = _df.to_dict(orient="records")

    if EXPORT_CSV:
        _p = str(_BASE / "motifs.csv")
        export_to_csv(_motif_list, _p)
        _export_paths["CSV"] = _p

    if EXPORT_BED:
        _p = str(_BASE / "motifs.bed")
        export_to_bed(_motif_list, _p)
        _export_paths["BED"] = _p

    if EXPORT_JSON:
        _p = str(_BASE / "motifs.json")
        export_to_json(_motif_list, _p)
        _export_paths["JSON"] = _p

    if EXPORT_EXCEL:
        _p = str(_BASE / "motifs.xlsx")
        export_to_excel(_motif_list, _p)
        _export_paths["Excel"] = _p

    # Per-class CSV sheets
    _class_dir = _BASE / "by_class"
    _class_dir.mkdir(exist_ok=True)
    for _cls, _grp in _df.groupby("Class"):
        _safe_cls = str(_cls).replace("/", "-").replace(" ", "_")
        _grp.to_csv(str(_class_dir / f"{_safe_cls}.csv"), index=False)
    print(f"📊 Per-class CSVs written to : {_class_dir}")

for _fmt, _path in _export_paths.items():
    print(f"   [{_fmt}] {_path}")

# ── 9. Per-sequence statistics table ─────────────────────────────────────────
_seq_table = pd.DataFrame([
    {
        "Sequence":       _sn[:60],
        "Length (bp)":    f"{_stats['length']:,}",
        "Motifs":         f"{_stats['motifs']:,}",
        "Density (m/kb)": f"{_stats['motifs'] / max(_stats['length'] / 1000, 0.001):.3f}",
        "Time (s)":       f"{_stats['elapsed_s']:.2f}",
        "Speed (kb/s)":   f"{_stats['throughput_kbps']:.1f}",
    }
    for _sn, _stats in _seq_stats.items()
])

print()
print("Per-sequence statistics:")
display(_seq_table)

# ── 10. Generate self-contained HTML report ───────────────────────────────────
import base64

def _img_tag(path: str, alt: str = "") -> str:
    """Embed image as base64 data URI for a self-contained HTML file."""
    with open(path, "rb") as _f:
        _b64 = base64.b64encode(_f.read()).decode()
    return f'<img src="data:image/png;base64,{_b64}" alt="{alt}" style="max-width:100%;margin:12px 0;"/>'

_class_rows = "".join(
    f"<tr><td>{_cls}</td><td style='text-align:right'>{_cnt:,}</td>"
    f"<td style='text-align:right'>{_cnt / max(_total_motifs, 1) * 100:.1f}%</td></tr>"
    for _cls, _cnt in sorted(_class_counts.items(), key=lambda x: -x[1])
)

_seq_rows = "".join(
    f"<tr><td>{_sn[:80]}</td>"
    f"<td style='text-align:right'>{_stats['length']:,}</td>"
    f"<td style='text-align:right'>{_stats['motifs']:,}</td>"
    f"<td style='text-align:right'>{_stats['motifs'] / max(_stats['length'] / 1000, 0.001):.3f}</td>"
    f"<td style='text-align:right'>{_stats['elapsed_s']:.2f}</td></tr>"
    for _sn, _stats in _seq_stats.items()
)

_fig_html = "".join(
    f"<h3>{_title}</h3>{_img_tag(_path, _title)}"
    for _title, _path in _fig_paths
)

_export_rows = "".join(
    f"<tr><td>{_fmt}</td><td><code>{Path(_path).name}</code></td></tr>"
    for _fmt, _path in _export_paths.items()
)

_len_html = ""
if _len_stats:
    _len_html = (
        f"<p><strong>Length (bp):</strong> "
        f"mean {_len_stats['mean']}, median {_len_stats['median']}, "
        f"min {_len_stats['min']}, max {_len_stats['max']}, "
        f"std {_len_stats['std']}</p>"
    )
_score_html = ""
if _score_stats:
    _score_html = (
        f"<p><strong>Score (1–3):</strong> "
        f"mean {_score_stats['mean']}, median {_score_stats['median']}, "
        f"min {_score_stats['min']}, max {_score_stats['max']}</p>"
    )

_html_report = textwrap.dedent(f"""\
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>NonBDNAFinder Report — {_RUN_TS}</title>
  <style>
    body   {{ font-family: 'Segoe UI', Arial, sans-serif; max-width: 1100px;
               margin: 0 auto; padding: 24px; color: #222; }}
    h1     {{ color: #1a237e; border-bottom: 2px solid #1a237e; padding-bottom: 6px; }}
    h2     {{ color: #283593; margin-top: 32px; }}
    h3     {{ color: #3949ab; margin-top: 24px; }}
    table  {{ border-collapse: collapse; width: 100%; margin: 12px 0; font-size: 0.92em; }}
    th     {{ background: #e8eaf6; padding: 7px 12px; text-align: left;
               border-bottom: 2px solid #9fa8da; }}
    td     {{ padding: 5px 12px; border-bottom: 1px solid #e0e0e0; }}
    tr:nth-child(even) td {{ background: #f5f5f5; }}
    .badge {{ display:inline-block; padding:3px 10px; border-radius:12px;
               background:#e8eaf6; font-size:0.85em; margin:3px; }}
    .stat  {{ display:inline-block; margin:6px 18px 6px 0; }}
    .stat span {{ font-size:1.7em; font-weight:700; color:#1a237e; }}
    code   {{ background:#f0f0f0; padding:1px 5px; border-radius:3px; }}
    footer {{ margin-top:48px; border-top:1px solid #e0e0e0; padding-top:12px;
               color:#888; font-size:0.85em; }}
  </style>
</head>
<body>

<h1>🧬 NonBDNAFinder — Non-B DNA Motif Analysis Report</h1>
<p>
  <strong>FASTA file:</strong> <code>{FASTA_FILE}</code> &nbsp;|
  <strong>Generated:</strong> {_RUN_TS} &nbsp;|
  <strong>Mode:</strong> {EXEC_MODE} &nbsp;|
  <strong>Chunk:</strong> {CHUNK_SIZE:,} bp
</p>

<h2>Executive Summary</h2>
<div>
  <div class="stat">Sequences<br/><span>{_n_seqs:,}</span></div>
  <div class="stat">Total bp<br/><span>{_total_bp:,}</span></div>
  <div class="stat">Total motifs<br/><span>{_total_motifs:,}</span></div>
  <div class="stat">Coverage<br/><span>{_coverage * 100:.1f}%</span></div>
  <div class="stat">Density (m/kb)<br/><span>{_density:.3f}</span></div>
  <div class="stat">Hybrids<br/><span>{_hybrid_count:,}</span></div>
  <div class="stat">Clusters<br/><span>{_cluster_count:,}</span></div>
  <div class="stat">Run time (s)<br/><span>{_total_elapsed:.1f}</span></div>
</div>
{_len_html}
{_score_html}

<h2>Adaptive Execution Plan</h2>
<table>
  <tr><th>Parameter</th><th>Value</th></tr>
  <tr><td>Total RAM</td><td>{_insp.get_total_ram() / 1e9:.2f} GB</td></tr>
  <tr><td>Available RAM</td><td>{_insp.get_available_ram() / 1e9:.2f} GB</td></tr>
  <tr><td>Memory budget</td><td>{_budget / 1e9:.2f} GB</td></tr>
  <tr><td>CPU cores</td><td>{_cpus}</td></tr>
  <tr><td>Chunk size</td><td>{CHUNK_SIZE:,} bp</td></tr>
  <tr><td>Overlap</td><td>{CHUNK_OVERLAP:,} bp</td></tr>
  <tr><td>Workers</td><td>{N_WORKERS}</td></tr>
  <tr><td>Mode</td><td>{EXEC_MODE}</td></tr>
</table>

<h2>Motifs by Class</h2>
<table>
  <tr><th>Class</th><th style='text-align:right'>Count</th><th style='text-align:right'>Fraction</th></tr>
  {_class_rows}
</table>

<h2>Per-Sequence Statistics</h2>
<table>
  <tr><th>Sequence</th><th style='text-align:right'>Length (bp)</th>
      <th style='text-align:right'>Motifs</th>
      <th style='text-align:right'>Density (m/kb)</th>
      <th style='text-align:right'>Time (s)</th></tr>
  {_seq_rows}
</table>

<h2>Figures</h2>
{_fig_html}

<h2>Exported Files</h2>
<table>
  <tr><th>Format</th><th>File</th></tr>
  {_export_rows}
</table>

<footer>
  NonBDNAFinder &mdash; Author: Dr. Venkata Rajesh Yella &mdash; License: MIT<br/>
  Report generated: {datetime.datetime.now().isoformat()}
</footer>
</body></html>
""")

_report_path = str(_BASE / "report.html")
with open(_report_path, "w", encoding="utf-8") as _rf:
    _rf.write(_html_report)

print()
print(f"📄 HTML report written to : {_report_path}")
print()
print("═" * 62)
print("  DONE — All outputs saved to:", str(_BASE))
print("═" * 62)

# Display clickable link inside the notebook
display(HTML(
    f'<p style="font-size:1.1em;">'
    f'✅ <strong>Report ready:</strong> '
    f'<a href="{_report_path}" target="_blank">open report.html</a></p>'
))